In [115]:
import cPickle as pickle
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
import pyLDAvis.gensim
import pyLDAvis

In [93]:
# f = file('lda.pkl', 'rb')
# loaded_obj = pickle.load(f)
lda = pickle.load(open('lda.p','rb'))
# f.close()
type(lda)

gensim.models.ldamodel.LdaModel

In [94]:
lda.print_topics()

[u'0.021*dr + 0.006*care + 0.005*one + 0.005*doctor + 0.004*see + 0.004*time + 0.004*get + 0.004*would + 0.003*like + 0.003*years',
 u"0.026*dr + 0.005*like + 0.005*great + 0.005*i've + 0.005*office + 0.004*he's + 0.004*would + 0.004*get + 0.004*doctor + 0.004*see",
 u'0.030*dr + 0.007*time + 0.006*great + 0.006*really + 0.005*staff + 0.005*would + 0.005*see + 0.005*office + 0.005*surgery + 0.005*doctor',
 u'0.025*dr + 0.006*would + 0.006*time + 0.005*back + 0.004*like + 0.004*surgery + 0.004*could + 0.004*get + 0.004*recommend + 0.004*staff',
 u'0.018*dr + 0.006*would + 0.005*like + 0.005*time + 0.004*surgery + 0.004*medical + 0.004*get + 0.004*office + 0.004*back + 0.004*one',
 u'0.018*dr + 0.011*doctor + 0.008*care + 0.006*time + 0.006*really + 0.005*great + 0.005*see + 0.005*like + 0.005*medical + 0.005*office',
 u'0.020*dr + 0.006*one + 0.005*like + 0.005*medical + 0.005*doctor + 0.005*get + 0.004*feel + 0.004*time + 0.004*great + 0.004*office',
 u'0.013*dr + 0.009*get + 0.008*off

In [95]:
# Create a new corpus & apply LDA model to the new corpus
# https://radimrehurek.com/gensim/models/ldamodel.html#id2
# https://radimrehurek.com/gensim/wiki.html#latent-dirichlet-allocation

new_document = [u"""We love Tamalpais Pediatrics and our children's doctor, John Lee. Like other reviewers have found, with their doctors, 
           Dr. Lee is patient, listens and addresses concerns we have about our childrens health.  I don't know if all pediatricians will come to the hospital to meet the new baby, but Dr. Lee has come to see both our daughters after they were born...
           We've seen Dr. Branco when Dr. Lee was unavailable and were relieved to see he and Dr. Lee are very similar in bedside manners.  
        Crystal Cox, the PA, is also a dream.  Shes warm and very easygoing with our young children.  I've no problems having her see the kids when one of the MD s are not available.
        I'm sorry to see Deana S. did not have a good experience with the receptionist, but I have to say that I've never had any problems with any of the staff.  They are all professional and very caring with the children.  
        5 Stars!  Thumbs up!"""]
# QUICK FORMAT: remove extra spaces and new lines
new_document = [" ".join(element.replace("\n","").split()) for element in new_doc]
#print(new_document)#type list, shape: (1,)

In [96]:
def canonicalize(word):    
    bads = ['dr.',"n't","'ve","'re",'wo',"'m"]
    goods = ['dr','not','have','are','will','am']

    try:
        idx = bads.index(word)
        word=goods[idx]
    except ValueError as e:
        word=word
    return word

In [97]:
# Defines the pattern for money and times - used by nltk tokenizer
def get_pat_for_tokenizer():
    from nltk.tokenize import regexp_tokenize
    """money and time patterns formatted to fit in nltk RegexpTokenizer object, debugged with s2 below"""
    #s2 = 'it took until 12:30 am to get an appointment and I spent $3.50'
    
    pat = '\w+|\$[\d\.]+|\S+'
    pat2 = '\d+\:[\d]+\s?(pm|am)|\S+'

    final_pattern = '('+pat2+')' +'|'+ '('+pat+')'#must be one string argument... | is OR
    return final_pattern

In [98]:
# NLTK code
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

tokenizer = RegexpTokenizer(get_pat_for_tokenizer())#time & money

# document obj passed to tokenizer must be type str
texts = [canonicalize(word) for word in tokenizer.tokenize(str(new_document).lower()) 
         if word not in stopwords.words()]
texts = [texts]#BOW dictionary requires texts obj be converted from type str to list
#print(texts)

In [99]:
# Store Dictionary
new_dictionary = corpora.Dictionary(texts)# create dictionary object from cleaned texts (texts type list obj)
new_dictionary.save('tamalpais_pediatrics.dict') # store the dictionary, for future reference
print(new_dictionary)
#print(new_dictionary.token2id)

Dictionary(69 unique tokens: [u'dream.', u'love', u'addresses', u"don't", u'doctor,']...)


In [143]:
# FORM new Bag-of-words corpus (count vectors)
# A trained LDA model can used be to transform new, unseen documents (formatted as BOW)

new_corpus = [new_dictionary.doc2bow(text) for text in texts]#both text and texts are type list

# Obtain LDA topic distributions for new BOW corpus:
#
# get topic probability distribution for a new review
# object: <gensim.interfaces.TransformedCorpus object
new_doc_lda = lda[new_corpus]
#new_doc_lda.print_topics()#doesn't work
#print(new_doc_lda)#This just prints the  at 0x10d3ccf10>
#print(lda.top_topics(corpus, num_words=5)[:][0])
#lda.top_topics(corpus, num_words=5)#shape (10,2) for num_words=5
#lda.print_topics(10)

# Obtain GAMMAS MATRIX:  topics by documents
# gammas =  (parameters controlling the topic weights) for each document
# Is normalization needed?
# throws away sstats, the second part of the returned tuple
gammas,_ = lda.inference(new_corpus)# first element is gammas matrix: 
norm_gammas = gammas/np.sum(gammas,axis=1)
#print(gammas)
#print(norm_gammas)

# PUT GAMMAS IN DATAFRAME, SORTED BY TOPIC IMPORTANCE
# Set row indices to match LDA topic numbers instead of being 0-indexed
df = pd.DataFrame(norm_gammas.T, index=np.arange(1,11))
#print(df)
print(df.sort(0,axis=0,ascending=False))

           0
5   0.532051
8   0.458309
6   0.001205
3   0.001205
10  0.001205
2   0.001205
7   0.001205
9   0.001205
4   0.001205
1   0.001205


In [104]:
# KEY QUESTIONS/To DO
# *** Still confused about exactly what to graph
# *** How do I get an example review that is highly "representative of a given topic?"
# I still don't know the best way to reduce the topic dimensonality space
# Work on resume
# Revise the Yelp API program and split into two programs - 1) get businesses, 2) get reviews

In [105]:
# row = [[d[1] for d in doc] for doc in corpus_lsi]
# df = pd.DataFrame(row).sort(0,axis=0,ascending=False)#0 sorts on the first column

In [114]:
# Visualization

vis_data = pyLDAvis.gensim.prepare(lda, new_corpus, new_dictionary)
pyLDAvis.display(vis_data)

AttributeError: 'Dictionary' object has no attribute 'id2word'